In [1]:
import os
os.chdir("../")
!pwd

/home/aditya/network_security


In [11]:
from pathlib import Path
from pydantic import BaseModel

class ModelEvaluationConfig(BaseModel):
    ## from config
    root_dir: Path
    test_data_path: Path
    metric_file_name: str
    target_column: str
    mlflow_uri: str
    ss_file_path: str
    model_run: str

In [12]:
from src.NetworkSecurity.constants import *
from src.NetworkSecurity.utils.common import read_yaml,create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self)->ModelEvaluationConfig:

        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        data_model_evaluation_config = ModelEvaluationConfig(

            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            mlflow_uri= config.mlflow_uri,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            ss_file_path = config.ss_file_path,
            model_run = config.model_run
        )

        return data_model_evaluation_config

In [13]:
cm = ConfigurationManager()
model_evaluation_config = cm.get_model_evaluation_config()

[2025-03-31 17:14:50,429: INFO: common : Yaml File: config/config.yaml loaded successfully]
[2025-03-31 17:14:50,433: INFO: common : Yaml File: params.yaml loaded successfully]
[2025-03-31 17:14:50,436: INFO: common : Yaml File: schema.yaml loaded successfully]
[2025-03-31 17:14:50,437: INFO: common : created directory at: artifacts]
[2025-03-31 17:14:50,437: INFO: common : created directory at: artifacts/model_evaluation]


In [16]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
from src.NetworkSecurity.utils.common import save_json
import pickle

class ModelEvaluate:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred, average='weighted')
        recall = recall_score(actual, pred, average='weighted')
        f1 = f1_score(actual, pred, average='weighted')
        return accuracy, precision, recall, f1 

    def validate_model(self, model_uri, sample_input):
        """Validates the model before using it for evaluation."""
        try:
            mlflow.models.predict(
                model_uri=model_uri,
                input_data=sample_input,
                env_manager="uv",
            )
            print(" Model validation successful!")
        except Exception as e:
            print(f"Model validation failed: {e}")
            raise e
    
    def evaluate(self):
        test_data = pd.read_csv(self.config.test_data_path)
        x_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        with open(self.config.ss_file_path, "rb") as file:
            scaler = pickle.load(file)
        x_test = scaler.transform(x_test)
        
        model_uri = f"runs:/{self.config.model_run}/model"  # Update with latest run ID
        loaded_model = mlflow.pyfunc.load_model(model_uri)

        # Validate model
        sample_input = x_test[:1]  # Taking a single row for validation
        self.validate_model(model_uri, sample_input)

        # Predict
        y_pred = loaded_model.predict(pd.DataFrame(x_test))
        # Evaluate
        accuracy, precision, recall, f1 = self.eval_metrics(y_test, y_pred)

        scores = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
        }

        save_json(self.config.metric_file_name,scores)
        print(f"Evaluation Metrics - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")


In [17]:
me = ModelEvaluate(model_evaluation_config)
me.evaluate()

2025/03/31 17:15:57 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/03/31 17:16:05 INFO mlflow.utils.virtualenv: Environment /tmp/tmpqw9wjurh/envs/virtualenv_envs/mlflow-e83e3d5a84004286d00d34d3eb4a7e3d743ccf22 already exists
2025/03/31 17:16:05 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /tmp/tmpqw9wjurh/envs/virtualenv_envs/mlflow-e83e3d5a84004286d00d34d3eb4a7e3d743ccf22/bin/activate && python -c ""']'
2025/03/31 17:16:05 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /tmp/tmpqw9wjurh/envs/virtualenv_envs/mlflow-e83e3d5a84004286d00d34d3eb4a7e3d743ccf22/bin/activate && python /home/aditya/miniconda3/envs/network_security/lib/python3.12/site-packages/mlflow/pyfunc/_mlflow_pyfunc_backend_predict.py --model-uri file:///tmp/tmp14m_h9ze/model --content-type json --input-path /tmp/tmpx_lhkwhg/input.json']'


{"predictions": [0]} Model validation successful!
[2025-03-31 17:16:08,218: INFO: common : json file saved at: artifacts/model_evaluation/metrics.json]
Evaluation Metrics - Accuracy: 0.9885, Precision: 0.9885383434094436, Recall: 0.9885, F1 Score: 0.9884991458924692
